In [1]:
import yaml
import os
import tensorflow as tf
from models import *
from datasets import *
import json
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

def load_config(config_path):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

def normalize_data(x_train, x_test):
    mean = np.mean(x_train, axis=0)
    std = np.std(x_train, axis=0)
    x_train_norm = (x_train - mean) / std
    x_test_norm = (x_test - mean) / std
    return x_train_norm, x_test_norm, mean, std

def main():
    # Load configurations
    regression_config = load_config('configs/regression.yaml')
    cnn1d_regression_config = load_config('configs/cnn1d_regression.yaml')
    print(regression_config)
    print(cnn1d_regression_config)

    # Initialize models
    regression_model = RegressionModel(regression_config)
    cnn1d_regression_model = CNN1DRegressionModel(cnn1d_regression_config)

    # Load dataset (using synthetic data for 1D CNN example)
    (x_train, y_train), (x_test, y_test) = load_dataset('boston_housing')

    # Normalize data
    x_train, x_test, x_mean, x_std = normalize_data(x_train, x_test)
    y_mean = np.mean(y_train)
    y_std = np.std(y_train)
    y_train = (y_train - y_mean) / y_std
    y_test = (y_test - y_mean) / y_std

    # Reshape data for 1D CNN (example only)
    x_train_cnn1d = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_test_cnn1d = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

    # Build models with input shape
    regression_model.build(x_train.shape[1:])
    cnn1d_regression_model.build(x_train_cnn1d.shape[1:])
    
    save_model_path = "results/saved_models"
    os.makedirs(save_model_path, exist_ok=True)  # Ensure the save directory exists

    # Compile and train regression model
    regression_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=regression_config['learning_rate']),
                             loss='mse',
                             metrics=['mae'])
    regression_model.fit(x_train, y_train, epochs=regression_config['epochs'], batch_size=regression_config['batch_size'])
    regression_model.save(os.path.join(save_model_path, 'regression.keras'))
    regression_save_config = regression_model.get_config()
    with open(os.path.join(save_model_path, 'regression_config.json'), 'w') as f:
        json.dump(regression_save_config, f)

    # Compile and train CNN1D regression model
    cnn1d_regression_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=cnn1d_regression_config['learning_rate']),
                                   loss='mse',
                                   metrics=['mae'])
    cnn1d_regression_model.fit(x_train_cnn1d, y_train, epochs=cnn1d_regression_config['epochs'], batch_size=cnn1d_regression_config['batch_size'])
    cnn1d_regression_model.save(os.path.join(save_model_path, 'cnn1d_regression.keras'))
    cnn1d_save_config = cnn1d_regression_model.get_config()
    with open(os.path.join(save_model_path, 'cnn1d_regression_config.json'), 'w') as f:
        json.dump(cnn1d_save_config, f)

    # Load models
    loaded_regression_model = tf.keras.models.load_model(
        os.path.join(save_model_path, 'regression.keras'), custom_objects={"RegressionModel": RegressionModel})
    loaded_cnn1d_regression_model = tf.keras.models.load_model(
        os.path.join(save_model_path, 'cnn1d_regression.keras'), custom_objects={"CNN1DRegressionModel": CNN1DRegressionModel})


    # Evaluate loaded models
    loaded_regression_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=regression_config['learning_rate']),
                                    loss='mse',
                                    metrics=['mae'])
    loss, mae = loaded_regression_model.evaluate(x_test, y_test)
    print(f'Loaded regression model evaluation - Loss: {loss}, MAE: {mae}')

    loaded_cnn1d_regression_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=cnn1d_regression_config['learning_rate']),
                                          loss='mse',
                                          metrics=['mae'])
    loss, mae = loaded_cnn1d_regression_model.evaluate(x_test_cnn1d, y_test)
    print(f'Loaded CNN1D regression model evaluation - Loss: {loss}, MAE: {mae}')

    # 예측 수행 및 결과 출력
    save_csv_path = "results/csv/"
    os.makedirs(save_csv_path, exist_ok=True)
    y_pred_regression = loaded_regression_model.predict(x_test)
    y_pred_cnn1d = loaded_cnn1d_regression_model.predict(x_test_cnn1d)

    # Un-normalize predictions
    y_pred_regression = y_pred_regression * y_std + y_mean
    y_pred_cnn1d = y_pred_cnn1d * y_std + y_mean
    y_test = y_test * y_std + y_mean

    # Calculate RMSE and R-squared for regression model
    rmse_regression = mean_squared_error(y_test, y_pred_regression, squared=False)
    r2_regression = r2_score(y_test, y_pred_regression)

    # Calculate RMSE and R-squared for CNN1D regression model
    rmse_cnn1d = mean_squared_error(y_test, y_pred_cnn1d, squared=False)
    r2_cnn1d = r2_score(y_test, y_pred_cnn1d)

    # Create DataFrame to store evaluation results
    results = pd.DataFrame({
        'True Values': y_test,
        'Regression Predictions': y_pred_regression.flatten(),
        'CNN1D Predictions': y_pred_cnn1d.flatten()
    })
    results['RMSE Regression'] = rmse_regression
    results['R2 Regression'] = r2_regression
    results['RMSE CNN1D'] = rmse_cnn1d
    results['R2 CNN1D'] = r2_cnn1d

    # Save results to a CSV file
    results.to_csv(os.path.join(save_csv_path, 'evaluation_results.csv'), index=False)

    print(results)

if __name__ == '__main__':
    main()


{'input_shape': [13], 'hidden_units': [64, 32], 'epochs': 10, 'batch_size': 32, 'learning_rate': 0.001}
{'input_shape': [128, 1], 'conv_layers': [{'filters': 32, 'kernel_size': 3, 'activation': 'relu'}, {'filters': 64, 'kernel_size': 3, 'activation': 'relu'}], 'dense_units': 64, 'epochs': 10, 'batch_size': 32, 'learning_rate': 0.001}


c:\Users\intol\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.3968 - mae: 0.8914
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8105 - mae: 0.6423  
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4649 - mae: 0.4569 
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3906 - mae: 0.3915 
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2663 - mae: 0.3502 
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2263 - mae: 0.3293 
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2098 - mae: 0.3049 
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1907 - mae: 0.2898 
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1403 - mae: 0.2521 
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1117 - mae: 0.2435 
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.9416 - mae: 0.7090
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7579 - mae: 0.6691
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 

c:\Users\intol\anaconda3\lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\intol\anaconda3\lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.0751 - mae: 0.7434  
Loaded regression model evaluation - Loss: 1.130346655845642, MAE: 0.7700888514518738
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.8906 - mae: 0.6842  
Loaded CNN1D regression model evaluation - Loss: 0.9631533026695251, MAE: 0.7175201773643494
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
     True Values  Regression Predictions  CNN1D Predictions  RMSE Regression  \
0            7.2               25.346943          23.302929         9.780208   
1           18.8               22.086683          22.790236         9.780208   
2           19.0               20.378542          23.037464         9.780208   
3           27.0               21.656462          24.511843         9.780208   
4           22.2               22.898540          23.093573         9.780208   
..           ...                     ...                ...              ...   
97          21.9               19.587240         

In [2]:
import yaml
import os
import tensorflow as tf
from models import *
from datasets import *
import json
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

def load_config(config_path):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

def main():
    # Load configurations
    regression_config = load_config('configs/regression.yaml')
    cnn1d_regression_config = load_config('configs/cnn1d_regression.yaml')
    print(regression_config)
    print(cnn1d_regression_config)

    # Initialize models
    regression_model = RegressionModel(regression_config)
    cnn1d_regression_model = CNN1DRegressionModel(cnn1d_regression_config)

    # Load dataset (using synthetic data for 1D CNN example)
    (x_train, y_train), (x_test, y_test) = load_dataset('boston_housing')

    # Reshape data for 1D CNN (example only)
    x_train_cnn1d = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_test_cnn1d = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

    # Build models with input shape
    regression_model.build(x_train.shape[1:])
    cnn1d_regression_model.build(x_train_cnn1d.shape[1:])
    
    save_model_path = "results/saved_models"
    os.makedirs(save_model_path, exist_ok=True)  # Ensure the save directory exists

    # Compile and train regression model
    regression_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=regression_config['learning_rate']),
                             loss='mse',
                             metrics=['mae'])
    regression_model.fit(x_train, y_train, epochs=regression_config['epochs'], batch_size=regression_config['batch_size'])
    regression_model.save(os.path.join(save_model_path, 'regression.keras'))
    regression_save_config = regression_model.get_config()
    with open(os.path.join(save_model_path, 'regression_config.json'), 'w') as f:
        json.dump(regression_save_config, f)

    # Compile and train CNN1D regression model
    cnn1d_regression_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=cnn1d_regression_config['learning_rate']),
                                   loss='mse',
                                   metrics=['mae'])
    cnn1d_regression_model.fit(x_train_cnn1d, y_train, epochs=cnn1d_regression_config['epochs'], batch_size=cnn1d_regression_config['batch_size'])
    cnn1d_regression_model.save(os.path.join(save_model_path, 'cnn1d_regression.keras'))
    cnn1d_save_config = cnn1d_regression_model.get_config()
    with open(os.path.join(save_model_path, 'cnn1d_regression_config.json'), 'w') as f:
        json.dump(cnn1d_save_config, f)

    # Load models
    loaded_regression_model = tf.keras.models.load_model(
        os.path.join(save_model_path, 'regression.keras'), custom_objects={"RegressionModel": RegressionModel})
    loaded_cnn1d_regression_model = tf.keras.models.load_model(
        os.path.join(save_model_path, 'cnn1d_regression.keras'), custom_objects={"CNN1DRegressionModel": CNN1DRegressionModel})


    # Evaluate loaded models
    loaded_regression_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=regression_config['learning_rate']),
                                    loss='mse',
                                    metrics=['mae'])
    loss, mae = loaded_regression_model.evaluate(x_test, y_test)
    print(f'Loaded regression model evaluation - Loss: {loss}, MAE: {mae}')

    loaded_cnn1d_regression_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=cnn1d_regression_config['learning_rate']),
                                          loss='mse',
                                          metrics=['mae'])
    loss, mae = loaded_cnn1d_regression_model.evaluate(x_test_cnn1d, y_test)
    print(f'Loaded CNN1D regression model evaluation - Loss: {loss}, MAE: {mae}')

    # 예측 수행 및 결과 출력
    save_csv_path = "results/csv/"
    os.makedirs(save_csv_path, exist_ok=True)
    y_pred_regression = loaded_regression_model.predict(x_test)
    y_pred_cnn1d = loaded_cnn1d_regression_model.predict(x_test_cnn1d)

    # Calculate RMSE and R-squared for regression model
    rmse_regression = mean_squared_error(y_test, y_pred_regression, squared=False)
    r2_regression = r2_score(y_test, y_pred_regression)

    # Calculate RMSE and R-squared for CNN1D regression model
    rmse_cnn1d = mean_squared_error(y_test, y_pred_cnn1d, squared=False)
    r2_cnn1d = r2_score(y_test, y_pred_cnn1d)

    # Create DataFrame to store evaluation results
    results = pd.DataFrame({
        'True Values': y_test,
        'Regression Predictions': y_pred_regression.flatten(),
        'CNN1D Predictions': y_pred_cnn1d.flatten()
    })
    results['RMSE Regression'] = rmse_regression
    results['R2 Regression'] = r2_regression
    results['RMSE CNN1D'] = rmse_cnn1d
    results['R2 CNN1D'] = r2_cnn1d

    # Save results to a CSV file
    results.to_csv(os.path.join(save_csv_path, 'evaluation_results.csv'), index=False)

    print(results)


if __name__ == '__main__':
    main()


{'input_shape': [13], 'hidden_units': [64, 32], 'epochs': 10, 'batch_size': 32, 'learning_rate': 0.001}
{'input_shape': [128, 1], 'conv_layers': [{'filters': 32, 'kernel_size': 3, 'activation': 'relu'}, {'filters': 64, 'kernel_size': 3, 'activation': 'relu'}], 'dense_units': 64, 'epochs': 10, 'batch_size': 32, 'learning_rate': 0.001}
Epoch 1/10


c:\Users\intol\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2488.2209 - mae: 42.2477
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 414.5920 - mae: 16.6828 
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 176.7500 - mae: 10.1440
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 93.4134 - mae: 8.0197  
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 69.6727 - mae: 5.3250 
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 71.4398 - mae: 6.4811  
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 57.8991 - mae: 5.1696 
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51.6760 - mae: 5.1570 
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 48.8677 - mae: 4.9212 
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 59.6542 - mae: 5.5043 
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 211.7507 - mae: 11.9977
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 174.1114 - mae: 10.8981
Epoch 3/10
13/13 ━━━━━━━━━━━━

c:\Users\intol\anaconda3\lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\intol\anaconda3\lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1018.8917 - mae: 27.1748  
Loaded regression model evaluation - Loss: 989.7361450195312, MAE: 26.361289978027344
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 320.1386 - mae: 15.0547  
Loaded CNN1D regression model evaluation - Loss: 343.90093994140625, MAE: 15.682975769042969
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/stepWARNING:tensorflow:6 out of the last 12 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000019F11521D80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_re